<a href="https://colab.research.google.com/github/CloseChoice/FlatCurver/blob/dev/data_analysis/notebooks/Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading 


In [0]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

url = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/Coronavirus.history.v2.csv'
url_pop = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/einwohner_bundeslaender.csv'
url_rki_neuinfektionen = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/RKI_Neuinfektionen_pro_land_pro_tag.csv'
url_rki_todesfaelle = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/RKI_Todesfaelle_pro_land_pro_tag.csv'
url_data = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/code/01_data%20acquisition/dataset.csv'

data = pd.read_csv(url)
df_ger = data.loc[data['parent'] == 'Deutschland'].copy()
df_infek = pd.read_csv(url_rki_neuinfektionen)
df_todesfaelle = pd.read_csv(url_rki_todesfaelle)
df = pd.read_csv(url_data)
df['Bundesweit:confirmed'] = df.filter(regex=(':confirmed$')).sum(axis=1)

pop = pd.read_csv(url_pop, sep='\t')


df_ger = pd.merge(df_ger, pop.set_index('Bundesland'), left_on='label', right_index=True, how='left')
df_ger = df_ger.loc[df_ger['label'] != 'Repatriierte', :].copy()



# Mapbox

In [0]:

fig = px.scatter_mapbox(df_ger, lat="lat", lon="lon", size="confirmed", animation_frame='date',
                  color_continuous_scale=px.colors.cyclical.IceFire)
fig.update_layout(mapbox_style="open-street-map").update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                'mapbox': {'pitch': 10, 'zoom': 5}})
fig.show()

In [0]:
fig = px.density_mapbox(df_ger, lat='lat', lon='lon', z='confirmed', radius=50, animation_frame='date', zoom=5, range_color=[0, max(df_ger['confirmed'])], height=800)
fig.update_layout(mapbox_style="open-street-map").update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                'mapbox': {'pitch': 10, 'zoom': 5}})
fig.show()

# Plots

In [0]:

fig = px.line(df_ger, x='date', y='confirmed', color='label')
fig.show()

In [0]:
fig = px.line(df_ger, x='date', y='confirmed', color='label', log_y=True)
fig.show()

# W.I.P.

In [0]:
df_bundesweit = df_ger.groupby('date').sum().reset_index()
fig = px.line(df_bundesweit, x='date', y='confirmed', log_y=True)
fig.show()

## TODO
* Add functions for marker
* Add Date to Mapbox

In [0]:
df_bundesweit = df_ger.groupby('date').sum().reset_index()

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "mapbox"}, {"type": "xy"}]])
fig.add_trace(go.Densitymapbox(lat=df_ger['lat'], lon=df_ger['lon'], z=df_ger['confirmed']), row=1, col=1).update_layout(mapbox_style="open-street-map", mapbox_center_lon=11,
                                                                                                                         mapbox_center_lat=50).update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                'mapbox': {'pitch': 10, 'zoom': 5}})
fig.add_trace(go.Scatter(x=df_bundesweit['date'], y=df_bundesweit['confirmed'], name='Bundesweit'), row=1, col=2)
fig.update_layout(height=800, showlegend=False, title_text='Bundesweite Entwicklung',
    updatemenus=[
        dict(active=0,
            buttons=list([
                dict(
                    args=[{'yaxis': {'type': 'linear'}}],
                    label="Linear",
                    method="relayout"
                ),
                dict(
                    args=[{'yaxis': {'type': 'log'}}],
                    label="Log",
                    method="relayout"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x=0,
            y=1.075,
        ),
    ]
)
fig.show()

# Exploration

**Outbreak**: First day with more than 100 national infections registered

In [0]:
fig = go.Figure()

outbreak = df.loc[df['Bundesweit:confirmed'] > 100].index.min()


for column in df.loc[df.index >= outbreak, :].filter(regex=(':confirmed$')).columns:
  fig.add_trace(go.Scatter(x=df.loc[df.index >= outbreak, :].index, y=df.loc[df.index >= outbreak, column], name=column))

fig.update_layout(height=800, title_text='Bundesweite Entwicklung',
    updatemenus=[
        dict(active=0,
            buttons=list([
                dict(
                    args=[{'yaxis': {'type': 'linear'}}],
                    label="Linear",
                    method="relayout"
                ),
                dict(
                    args=[{'yaxis': {'type': 'log'}}],
                    label="Log",
                    method="relayout"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x=0,
            y=1.075,
        ),
    ]
)
fig.show()

### Calculate daily change since outbreak

In [0]:
df_outbreak = df.loc[df.index >= outbreak, :].filter(regex=(':confirmed$')).copy()

for column in df_outbreak:
  df_outbreak[column+':change'] = df[column] / df[column].shift(1)

In [0]:
fig = go.Figure()

for column in df_outbreak.filter(regex=(':change$')).columns:
  fig.add_trace(go.Scatter(x=df_outbreak.index, y=df_outbreak[column], name=column))

fig.update_layout(height=800, title_text='Bundesweite Entwicklung',
    updatemenus=[
        dict(active=0,
            buttons=list([
                dict(
                    args=[{'yaxis': {'type': 'linear'}}],
                    label="Linear",
                    method="relayout"
                ),
                dict(
                    args=[{'yaxis': {'type': 'log'}}],
                    label="Log",
                    method="relayout"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x=0,
            y=1.075,
        ),
    ]
)

fig.show()

## Average since outbreak


If infection rate does not change, every day leads to 10M infections in 21 days

In [0]:
static_change = df_outbreak['Bundesweit:confirmed:change'].mean()

def next_days(days=21, column='Bundesweit:confirmed', static_change=static_change):
  days_since_outbreak = df_outbreak.index.max()
  status_quo = df_outbreak.loc[days_since_outbreak, column]
  infections = []
  for i in range(days):
    status_quo *= static_change
    infections.append(int(status_quo))

  return pd.Series(infections, index=range(days_since_outbreak+1, days_since_outbreak+days+1))

df_future = df_outbreak['Bundesweit:confirmed'].copy()
df_future_a = df_future.append(next_days())
df_future_b = df_future.append(next_days(static_change=static_change*0.9))


  

In [0]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_future_a.index, y=df_future_a.values, name='Entwicklung ohne Massnahmen'))
fig.add_trace(go.Scatter(x=df_future_b.index, y=df_future_b.values, name='Reduktion der Inektionsrate um 10%'))
fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=53,
            y0=0,
            x1=53,
            y1=df_future.max(),
            line=dict(
                color="RoyalBlue",
                width=3
            ))
        )
fig.update_layout(height=800, title_text='Bundesweite Entwicklung',
    updatemenus=[
        dict(active=0,
            buttons=list([
                dict(
                    args=[{'yaxis': {'type': 'linear'}}],
                    label="Linear",
                    method="relayout"
                ),
                dict(
                    args=[{'yaxis': {'type': 'log'}}],
                    label="Log",
                    method="relayout"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x=0,
            y=1.075,
        ),
    ]
)
fig.show()